In [ ]:
# ─── 0) 시스템·파이썬 패키지 설치 ───────────────────────────────────────
%%bash
apt-get update -y
apt-get install -y ffmpeg unzip

pip install --upgrade yt-dlp faster-whisper soundfile tqdm pandas bs4 ffmpeg-python


Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,059 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,036 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,750

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.


In [ ]:
# ─── 1) TwitchDownloaderCLI 설치 ───────────────────────────────────────
%%bash
wget -q https://github.com/lay295/TwitchDownloader/releases/download/1.55.0/TwitchDownloaderCLI-1.55.0-Linux-x64.zip
unzip -q TwitchDownloaderCLI-1.55.0-Linux-x64.zip
chmod +x TwitchDownloaderCLI


In [ ]:
# ─── 2) 설정 값 정의 ───────────────────────────────────────────────────
VOD_ID    = "2490741688"  # 분석할 VOD ID 여기를 바꿔주기
VOD_URL   = f"https://www.twitch.tv/videos/{VOD_ID}"
AUDIO_M4A = f"/content/{VOD_ID}.m4a"
WAV_PATH  = f"/tmp/{VOD_ID}.wav"
CHAT_HTML = f"/content/{VOD_ID}_chat.html"


In [ ]:
# ─── 3) 오디오 추출 (.m4a) ─────────────────────────────────────────────
# Step 3: in a Python cell
!yt-dlp --extract-audio --audio-format m4a \
  -o "{AUDIO_M4A}" "{VOD_URL}"


[twitch:vod] Extracting URL: https://www.twitch.tv/videos/2490741688
[twitch:vod] 2490741688: Downloading stream metadata GraphQL
[twitch:vod] 2490741688: Downloading video access token GraphQL
[twitch:vod] 2490741688: Downloading m3u8 information
[twitch:vod] 2490741688: Downloading storyboard metadata JSON
[info] v2490741688: Downloading 1 format(s): Audio_Only
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 1841
[download] Destination: /content/2490741688.mp4
[download] 100% of  773.56MiB in 00:14:23 at 917.63KiB/s
[FixupM3u8] Fixing MPEG-TS in MP4 container of "/content/2490741688.mp4"
[ExtractAudio] Destination: /content/2490741688.m4a
Deleting original file /content/2490741688.mp4 (pass -k to keep)


In [ ]:
# ─── 4) Twitch 채팅 HTML 다운로드 ─────────────────────────────────────────────
# (Python 셀에 ! 접두어)
!./TwitchDownloaderCLI chatdownload --id {VOD_ID} -o "{CHAT_HTML}"


=TwitchDownloaderCLI 1.55.0 Copyright (c) lay295 and contributors
The file '/content/2490741688_chat.html' already exists.
[O] Overwrite / [R] Rename / [E] Exit: o
[STATUS] - Downloading 100%
[STATUS] - Writing Output File


In [ ]:
import glob
print(glob.glob("/content/*.html"))


['/content/2490741688_chat.html']


In [ ]:
# ─── 5) M4A → WAV 변환 (16 kHz, mono) ────────────────────────────────────
import subprocess

proc = subprocess.run(
    ["ffmpeg", "-y", "-i", AUDIO_M4A, "-ar", "16000", "-ac", "1", WAV_PATH],
    capture_output=True, text=True
)
if proc.returncode != 0:
    print("FFmpeg error:\n", proc.stderr)
    raise RuntimeError("오디오 변환 실패")
print("✅ WAV 변환 완료:", WAV_PATH)


✅ WAV 변환 완료: /tmp/2490741688.wav


In [ ]:
# ─── 6) WhisperX 설치 (Colab 셀에 !) ──────────────────────────────────────
!pip install git+https://github.com/m-bain/whisperx.git soundfile


  Cloning https://github.com/m-bain/whisperx.git to /tmp/pip-req-build-18t9dutz
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperx.git /tmp/pip-req-build-18t9dutz
  Resolved https://github.com/m-bain/whisperx.git to commit b93e9b6f575f41017c01c51d932f756fedadcde4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install --upgrade whisperx soundfile


In [ ]:
!pip install faster-whisper soundfile tqdm


In [ ]:
#----------------------7 transcript----------------
import soundfile as sf
import math
import pandas as pd
from tqdm.notebook import tqdm
from faster_whisper import WhisperModel

# 1) WAV 파일 불러오기 (이미 16kHz, 모노로 변환된 상태여야 합니다)
audio, sr = sf.read(WAV_PATH, dtype="float32")

# 2) 모델 로드 (tiny 모델, CPU 모드, int8 양자화)
model = WhisperModel("tiny", device="cpu", compute_type="int8")

# 3) 청크 길이 및 개수 계산
chunk_sec  = 120                                  # 한 청크당 120초
chunk_size = int(chunk_sec * sr)                  # 프레임 단위
num_chunks = math.ceil(len(audio) / chunk_size)   # 총 청크 개수

all_segments = []

# 4) tqdm 진행바와 함께 청크별로 전사 실행
for idx in tqdm(range(num_chunks), desc="전사 중", unit="chunk"):
    start = idx * chunk_size
    end   = min(start + chunk_size, len(audio))
    chunk = audio[start:end]

    # faster-whisper의 transcribe는 VAD 없이 바로 디코딩
    segments, _ = model.transcribe(
        chunk,
        beam_size=1,    # greedy 서치
        language="en"   # 영어 자막일 경우
    )

    # 시간 오프셋 보정해서 저장
    offset = start / sr
    for seg in segments:
        all_segments.append({
            "start": seg.start + offset,
            "end":   seg.end   + offset,
            "text":  seg.text
        })

# 5) 결과를 DataFrame으로 정리
df = pd.DataFrame(all_segments)
df = df.sort_values("start").reset_index(drop=True)

# 보기 좋게 HH:MM:SS.mmm 포맷으로 변경
def format_ts(t):
    h, rem = divmod(t, 3600)
    m, s   = divmod(rem, 60)
    return f"{int(h):02d}:{int(m):02d}:{s:06.3f}"

df["time"] = df["start"].apply(format_ts)
transcript_df = df[["time", "text"]]

# 샘플 출력
print(transcript_df.head(), "\n… 총", len(transcript_df), "행")


tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.25k [00:00<?, ?B/s]

vocabulary.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

전사 중:   0%|          | 0/154 [00:00<?, ?chunk/s]

           time                   text
0  00:00:00.000                    You
1  00:00:30.000                    You
2  00:00:42.000               Hey chat
3  00:00:55.000      Notifications off
4  00:00:57.000   What does that mean? 
… 총 2616 행


In [ ]:
# ─── 8) Twitch 채팅 HTML 파싱 → DataFrame ────────────────────────────────
from bs4 import BeautifulSoup

def parse_twitch_comments(path: str) -> pd.DataFrame:
    with open(path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    data = []
    for pre in soup.find_all("pre", class_="comment-root"):
        # 시간
        t = pre.text.split("]")[0].strip("[")
        # 배지
        badge = pre.find("img", class_="badge-image")
        badge_html = f'<img src="{badge["src"]}" width="20">' if badge else ""
        # 작성자
        auth = pre.find("span", class_="comment-author")
        author = auth.text if auth else ""
        author_html = (author + " " + badge_html).strip()
        # 메시지 + 이모트        <-- 여기 이모트를 따로 뺄수있는지?
        msg_tag = pre.find("span", class_="comment-message")
        msg = msg_tag.text.strip(": ") if msg_tag else ""
        for em in pre.find_all("img", class_="emote-image"):
            msg += f' <img src="{em["src"]}" width="20">'
        data.append({"time": t, "author": author_html, "message": msg})
    return pd.DataFrame(data)

chat_df = parse_twitch_comments(CHAT_HTML)
print("=== Chat DataFrame ===")
print(chat_df.head(), "\n… 총", len(chat_df), "행")


=== Chat DataFrame ===
      time                                             author  \
0  0:00:04  KittyKingTV <img src="https://static-cdn.jtvnw...   
1  0:00:06  KittyKingTV <img src="https://static-cdn.jtvnw...   
2  0:00:17  Kman2see <img src="https://static-cdn.jtvnw.ne...   
3  0:00:18  Kman2see <img src="https://static-cdn.jtvnw.ne...   
4  0:00:20  guactoast <img src="https://static-cdn.jtvnw.n...   

                                             message  
0  Live <img src="https://cdn.7tv.app/emote/01F7K...  
1  Pog <img src="https://cdn.betterttv.net/franke...  
2                                               LIVE  
3  Pog <img src="https://cdn.betterttv.net/franke...  
4  Pog LIVE <img src="https://cdn.betterttv.net/f...   
… 총 14421 행


In [ ]:
transcript_df

,time,text,time_delta,second,td,sec
0,00:00:00.000,You,0 days 00:00:00,0 days 00:00:00,0 days 00:00:00,0
1,00:00:30.000,You,0 days 00:00:30,0 days 00:00:30,0 days 00:00:30,30
2,00:00:42.000,Hey chat,0 days 00:00:42,0 days 00:00:42,0 days 00:00:42,42
3,00:00:55.000,Notifications off,0 days 00:00:55,0 days 00:00:55,0 days 00:00:55,55
4,00:00:57.000,What does that mean?,0 days 00:00:57,0 days 00:00:57,0 days 00:00:57,57
...,...,...,...,...,...,...
2611,05:06:13.840,aimed really hard at the wall because you jus...,0 days 05:06:13.840000,0 days 05:06:13,0 days 05:06:13.840000,18373
2612,05:06:18.720,"But this one, this motherfucker, it's like if...",0 days 05:06:18.720000,0 days 05:06:18,0 days 05:06:18.720000,18378
2613,05:06:27.280,"fucking shallow as possible, it's insane, it'...",0 days 05:06:27.280000,0 days 05:06:27,0 days 05:06:27.280000,18387
2614,05:06:34.240,"gonna shut up at end stream, that was, that w...",0 days 05:06:34.240000,0 days 05:06:34,0 days 05:06:34.240000,18394


In [ ]:
chat_df

,time,author,message,emote_names,sec,td,chat_idx
0,0:00:04,"KittyKingTV <img src=""https://static-cdn.jtvnw...",Live,,4,0 days 00:00:04,1
1,0:00:06,"KittyKingTV <img src=""https://static-cdn.jtvnw...",Pog,,6,0 days 00:00:06,1
2,0:00:17,"Kman2see <img src=""https://static-cdn.jtvnw.ne...",LIVE,,17,0 days 00:00:17,1
3,0:00:18,"Kman2see <img src=""https://static-cdn.jtvnw.ne...",Pog,,18,0 days 00:00:18,1
4,0:00:20,"guactoast <img src=""https://static-cdn.jtvnw.n...",Pog LIVE,,20,0 days 00:00:20,1
...,...,...,...,...,...,...,...
14416,5:06:42,"LordFartamor <img src=""https://static-cdn.jtvn...",lordfa10PREGAMOR,,18402,0 days 05:06:42,2
14417,5:06:42,"GGfeima <img src=""https://static-cdn.jtvnw.net...",one last time,,18402,0 days 05:06:42,3
14418,5:06:42,RainerPb,@ClintStevens heres the clip for you after str...,,18402,0 days 05:06:42,4
14419,5:06:42,"KingHarpo <img src=""https://static-cdn.jtvnw.n...",GGS,,18402,0 days 05:06:42,5


In [ ]:
import pandas as pd
import time

# 1) sec 키 생성 (밀리초 버림)
transcript_df['sec'] = pd.to_timedelta(transcript_df['time']).dt.floor('S') \
                                       .dt.total_seconds().astype(int)
chat_df['sec']       = pd.to_timedelta(chat_df['time']).dt.floor('S') \
                                       .dt.total_seconds().astype(int)

# 2) badge_img 분리 + 순수 작성자 이름만 남기기
chat_df['badge_img']      = chat_df['author'] \
    .str.extract(r'<img[^>]*src="([^"]+)"')[0] \
    .fillna('')
chat_df['chat_authors']   = chat_df['author'] \
    .str.replace(r'\s*<img[^>]*>', '', regex=True)

# 3) 같은 초(sec)의 자막을 합친 맵 생성
caption_map = transcript_df.groupby('sec')['text'] \
    .agg(" | ".join) \
    .to_dict()

# 4) chat_df에 caption 매핑
chat_df['caption'] = chat_df['sec'].map(caption_map)

# 5) 정렬 및 중복 캡션 제거 (같은 sec에서 첫 행만 남기기)
chat_df = chat_df.sort_values(['sec','time'])
chat_df['is_first'] = chat_df.groupby('sec').cumcount() == 0
chat_df.loc[~chat_df['is_first'], 'caption'] = ''

# 6) 최종 DataFrame 구성
final_df = chat_df.rename(columns={
    'message':     'chat_messages',
    'emote_names': 'emoticon_names'
})[[
    'time',
    'caption',
    'chat_authors',
    'badge_img',
    'chat_messages',
    'emoticon_names'
]]

# 7) CSV 저장 및 확인
out_path = '/content/merged_with_badges.csv'
final_df.to_csv(out_path, index=False)

# 간단히 출력
print(final_df.head(10))
print(f"\n✅ merged_with_badges.csv saved at: {out_path}")


/tmp/ipython-input-45-3911222590.py:5: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  transcript_df['sec'] = pd.to_timedelta(transcript_df['time']).dt.floor('S') \
/tmp/ipython-input-45-3911222590.py:7: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  chat_df['sec']       = pd.to_timedelta(chat_df['time']).dt.floor('S') \


      time caption chat_authors  \
0  0:00:04     NaN  KittyKingTV   
1  0:00:06     NaN  KittyKingTV   
2  0:00:17     NaN     Kman2see   
3  0:00:18     NaN     Kman2see   
4  0:00:20     NaN    guactoast   
5  0:00:20          CrxstalStar   
6  0:00:21     NaN   Chinstero_   
7  0:00:22     NaN  CrxstalStar   
8  0:00:24     NaN    MHanDaMan   
9  0:00:25     NaN     Kman2see   

                                           badge_img        chat_messages  \
0  https://static-cdn.jtvnw.net/badges/v1/4b76d5f...                 Live   
1  https://static-cdn.jtvnw.net/badges/v1/4b76d5f...                  Pog   
2  https://static-cdn.jtvnw.net/badges/v1/b817aba...                 LIVE   
3  https://static-cdn.jtvnw.net/badges/v1/b817aba...                  Pog   
4  https://static-cdn.jtvnw.net/badges/v1/bbbe0db...             Pog LIVE   
5  https://static-cdn.jtvnw.net/badges/v1/4300a89...                  yo-   
6  https://static-cdn.jtvnw.net/badges/v1/c03962e...                 PogU  